In [ ]:
%pip install --upgrade -r requirements.txt

Basic Anthropic Integration

In [ ]:
from anthropic import Anthropic 
from payi.lib.helpers import payi_anthropic_url
import json
import os

# Read the API KEYs from the environment, replace the default values (the second argument) with your own keys if needed
anthropic_key = os.getenv("ANTHROPIC_API_KEY", "YOUR_ANTHROPIC_API_KEY")
payi_api_key = os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

payi_headers = {
    "xProxy-api-key": payi_api_key,
}

client = Anthropic(
    base_url=payi_anthropic_url(),
    api_key=anthropic_key,
    default_headers=payi_headers
)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ]
)
completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))


Handle streaming calls. xproxy_result is returned as part of the last chunk.

In [ ]:
stream = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    stream=True
)

message = ""
input_tokens = None
output_tokens = None
for event in stream:
    match event.type:
        case "message_start":
            input_tokens = event.message.usage.input_tokens
        case "content_block_start":
            message += event.content_block.text
        case "content_block_delta":
            message += event.delta.text
        case "message_delta":
            output_tokens = event.usage.output_tokens
            if 'xproxy_result' in event.model_extra:
                xproxy_result = event.model_extra['xproxy_result']
        case "content_block_stop" | "message_stop":
            ...

print(message)
if (xproxy_result is not None):
    print()
    print(json.dumps(xproxy_result, indent=4))

Use the Pay-i SDK to send a request with request tags

In [ ]:
from payi.lib.helpers import create_headers

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a request tag test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(request_tags=["x", "y"])
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

Create a limit and make a request with that limit

In [ ]:
from payi import Payi

payi_client = Payi(
    api_key=payi_api_key
)

#Create a limit
limit_response = payi_client.limits.create(
    #As long as the limit configuration remains the same across creates, the same limit name can be used repeatedly
    limit_name='Anthropic quickstart allow limit',
    max=12.50, #$12.50 USD
    limit_type="Allow",
    limit_tags=["example_limit"]
)

limit_name = limit_response.limit.limit_name
limit_id = limit_response.limit.limit_id

print("Limit Created")
print(f"Limit Name: {limit_name}")
print(f"Limit ID: {limit_id}")

#Make a request using the new limit and request tags
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        request_tags=["x", "y"],
        limit_ids=[limit_id]
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

See limit status

In [ ]:
response = payi_client.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.output.base}")

Make an ingest call with pre-computed token values

In [ ]:
response = payi_client.ingest.units(
    category="system.anthropic",
    resource="claude-3-5-sonnet-20241022",
    units={ "text": { "input": 50, "output": 100 } },
    limit_ids=[limit_id],
    request_tags=["a", "b"]
)

print(f"Ingest request ID: {response.request_id}")
print(f"Input Base Cost: {response.xproxy_result.cost.input.base}")
print(f"Output Base Cost: {response.xproxy_result.cost.output.base}")
print(f"Total Base Cost: {response.xproxy_result.cost.total.base}")

Reset a limit back to zero tracked cost

In [ ]:
response = payi_client.limits.reset(limit_id=limit_id)
print(response.message)
print("State prior to reset: ")
print(f"Limit Name: {response.limit_history.limit_name}")
print(f"Limit ID: {response.limit_history.limit_id}")
print(f"Limit Tags: {response.limit_history.limit_tags}")
print(f"Limit Reset Timestamp: {response.limit_history.limit_reset_timestamp}")
print(f"Limit Input Base Cost: {response.limit_history.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit_history.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi_client.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")

Create a small blocking limit that will prevent calls from happening that exceed the maximum, then capture the output.

In [ ]:
limit_response = payi_client.limits.create(
    #As long as the limit configuration remains the same across creates, the same limit name can be used repeatedly
    limit_name='Anthropic quickstart block limit',
    max=0.00000001, 
    limit_type="block",
    limit_tags=["limit_block_example"]
)
block_limit = limit_response.limit.limit_id

print("Limit Created")
print(f"Limit Name: {limit_response.limit.limit_name}")
print(f"Limit ID: {limit_response.limit.limit_id}")

try:
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "provide me a list of toys for children 5 and under"
                    }
                ]
            }
        ],
        extra_headers=create_headers(
            request_tags=["x", "y"],
            limit_ids=[block_limit]
        )
    )

    completion = response.content[0].text
    print(completion)

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=500,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "tell me a short story about a toy"
                    }
                ]
            }
        ],
        extra_headers=create_headers(
            request_tags=["x", "y"],
            limit_ids=[block_limit]
        )
    )
except Exception as e:
    print(json.dumps(e.body, indent=4))

Create an experience type and send a request with the experience type. Pay-i will auto generate an experience id that can be specified later.

In [ ]:
# Create an experience type
exp_name="quickstart_experience"
exp_type_response = payi_client.experiences.types.create(
    name=exp_name,
    description="An example of an experience"
)

# Make a request using the limit, request tags, and experience
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        request_tags=["x", "y"],
        limit_ids=[limit_id],
        experience_name=exp_name
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
experience_id = xproxy_result['experience_id']
print(json.dumps(xproxy_result, indent=4))

Send a request with a limit and user ID

In [ ]:
# Make a request using the limit, request tags, and user id
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=500,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Say 'this is a test'"
                }
            ]
        }
    ],
    extra_headers=create_headers(
        limit_ids=[limit_id],
        # user id can be any string value
        user_id="example_user_id"
    )
)

completion = message.content[0].text
print(completion)

xproxy_result = message.xproxy_result
print(json.dumps(xproxy_result, indent=4))

List and then delete all limits

In [ ]:
response = payi_client.limits.list()
for limit in response.items:
    print("Deleting limit with id:" + limit.limit_id)
    payi_client.limits.delete(limit.limit_id)